<a href="https://colab.research.google.com/github/WengGeyu/NMA-DL/blob/main/data_to_nnUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/ColabNotebooks')
#!git clone https://github.com/MIC-DKFZ/nnUNet.git
%cd nnUNet
!pip install -e .
!pip install --upgrade git+https://github.com/FabianIsensee/hiddenlayer.git

In [ ]:
from batchgenerators.utilities.file_and_folder_operations import *
import shutil
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json
from nnunetv2.paths import nnUNet_raw

def convert_aneurysm(kits_base_dir: str, nnunet_dataset_id: int = 666):
    task_name = "aneurysm"

    foldername = "Dataset%03.0d_%s" % (nnunet_dataset_id, task_name)

    # setting up nnU-Net folders
    out_base = join('/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_raw', foldername)
    imagestr = join(out_base, "imagesTr")
    labelstr = join(out_base, "labelsTr")
    maybe_mkdir_p(imagestr)
    maybe_mkdir_p(labelstr)

    cases = subdirs(kits_base_dir, prefix='sub_', join=False)
    for tr in cases:
        shutil.copy(join(kits_base_dir, tr, 'angio.nii.gz'), join(imagestr, f'{tr}_0000.nii.gz'))
        shutil.copy(join(kits_base_dir, tr, 'Lesion_1_mask.nii.gz'), join(labelstr, f'{tr}.nii.gz'))

    generate_dataset_json(out_base, {0: "TOF"},
                          labels={
                              "background": 0,
                              "aneurysm": 1
                          },
                          num_training_cases=len(cases), file_ending='.nii.gz')

convert_aneurysm('/content/drive/MyDrive/ColabNotebooks/data', 666)

In [ ]:
! nnUNet_raw="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_raw" \
nnUNet_preprocessed="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_preprocessed" \
nnUNet_results="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_results" \
nnUNetv2_plan_and_preprocess -d 666 --verify_dataset_integrity

Training

In [25]:
! nnUNet_raw="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_raw" \
nnUNet_preprocessed="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_preprocessed" \
nnUNet_results="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_results" \
nnUNetv2_train 666 3d_lowres 0 -tr nnUNetTrainer_10epochs --c

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


This is the configuration used by this training:
Configuration name: 3d_lowres
 {'data_identifier': 'nnUNetPlans_3d_lowres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 2, 'patch_size': [56, 224, 192], 'median_image_size_in_voxels': [93, 335, 294], 'spacing': [0.8319214816658427, 0.6203981293351043, 0.6203981293351043], 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [False], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': [2, 2, 2, 2, 2, 2], 'n_conv_per_stage_decoder': [2, 2, 2, 2, 2], 

In [24]:
! nnUNet_raw="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_raw" \
nnUNet_preprocessed="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_preprocessed" \
nnUNet_results="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_results" \
nnUNetv2_predict -i INPUT -o OUTPUT -d 666 -c 2d -device cuda -f 0
# -chk checkpoint_best.pth


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

usage: nnUNetv2_predict
       [-h]
       -i
       I
       -o
       O
       -d
       D
       [-p P]
       [-tr TR]
       -c
       C
       [-f F [F ...]]
       [-step_size STEP_SIZE]
       [--disable_tta]
       [--verbose]
       [--save_probabilities]
       [--continue_prediction]
       [-chk CHK]
       [-npp NPP]
       [-nps NPS]
       [-prev_stage_predictions PREV_STAGE_PREDICTIONS]
       [-num_parts NUM_PARTS]
       [-part_id PART_ID]
       [-device DEVICE]
nnUNetv2_predict: error: the following arguments are required: -i, -o, -c


In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

input_path = "/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_results/Dataset666_aneurysm/nnUNetTrainer__nnUNetPlans__2d/fold_0/validation"
input_filename = os.path.join(input_path,'sub_013.nii.gz')
image_obj = nib.load(input_filename)
# Extract data as numpy ndarray
image_data = image_obj.get_fdata()
# Get the image shape
height, width, depth = image_data.shape
# print(f"The image object has the following dimensions: height: {height}, width:{width}, depth:{depth}")
# Select random layer number
maxval = depth
i = np.random.randint(0, maxval)
# Define a channel to look at
print(f"Plotting Layer {i} of Image")
plt.imshow(image_data[:, :, i], cmap='gray')
plt.axis('off');